# Motion Detection and Tracking
Uses all the things we have learnt so far
- operations on images/frames
- blur
- threshold
- dilate
- contours

In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture('./Videos/vtest.avi')

ret, frame1 = cap.read()
ret, frame2 = cap.read()

In [3]:
while cap.isOpened():
    
    # find the diff of the two frames
    diff = cv2.absdiff(frame1, frame2)
    
    # convert it to grayscale mode: easier to work with graysccale images
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    # blur the image: remove the noises
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    # find the threshold of the image
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    
    # dilate the image: to eliminate small threshold lines
    dilated = cv2.dilate(thresh, None, iterations=3)
    
    # find contours
    image, contours, hierarchy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 900:
            continue
        cv2.rectangle(frame1, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame1, 'Status: {}'.format('Movement'), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
    
    # show the window
    cv2.imshow('feed', frame1)
    frame1 = frame2
    ret, frame2 = cap.read()
    
    # exiting the window
    if cv2.waitKey(40) == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

# Detect Simple Geometric Shapes

In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('./Pictures/shapes.png')
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_,thresh = cv2.threshold(imgGray, 240, 255, cv2.THRESH_BINARY)
_,contours,_ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [3]:
for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)
    cv2.drawContours(img, [approx], 0, (0,0,0), 5)
    x = approx.ravel()[0]
    y = approx.ravel()[1]
    if len(approx) == 3:
        cv2.putText(img, 'Triangle', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    elif len(approx) == 4:
        x1,y1,w,h = cv2.boundingRect(approx)
        aspectRatio = float(w)/h
        if aspectRatio >= 0.95 and aspectRatio <= 1.05:
            cv2.putText(img, 'Square', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
        else:
            cv2.putText(img, 'Rectangle', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    elif len(approx) == 5:
        cv2.putText(img, 'Pentagon', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    elif len(approx) == 10:
        cv2.putText(img, 'Star', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))
    else:
        cv2.putText(img, 'Circle', (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0))

In [4]:
cv2.imshow('shapes', img)
cv2.waitKey(0)
cv2.destroyAllWindows()